# 如何添加对话历史摘要

持久性最常见的用例之一是用它来跟踪对话历史记录。这太棒了——它让继续对话变得容易。然而，随着对话变得越来越长，此对话历史记录可能会累积并占用越来越多的上下文窗口。这通常是不可取的，因为它会导致对 LLM 的调用更加昂贵和耗时，并且可能会出现错误。解决这个问题的一种方法是创建迄今为止的对话摘要，并将其与过去的 N 条消息一起使用。本指南将通过一个示例来说明如何执行此操作。

这将涉及几个步骤：
- 检查对话是否太长（可以通过检查消息数量或消息长度来完成）
- 如果是，则创建摘要（需要提示）
- 然后删除除最后 N 条消息之外的所有消息

其中很大一部分是删除旧消息。有关如何执行此操作的深入指南，请参阅[本指南](../delete-messages)

## 设置

首先，让我们设置我们要使用的包
```bash
npm install @langchain/langgraph @langchain/anthropic @langchain/core uuid
```
接下来，我们需要为 Anthropic 设置 API 密钥（我们将使用的 LLM）
```typescript
process.env.ANTHROPIC_API_KEY = 'YOUR_API_KEY'
```
或者，我们可以为 [LangSmith 追踪](https://smith.langchain.com/) 设置 API 密钥，这将为我们提供一流的可观察性。
```typescript
process.env.LANGCHAIN_TRACING_V2 = 'true'
process.env.LANGCHAIN_API_KEY = 'YOUR_API_KEY'
```

## 构建聊天机器人

现在让我们构建聊天机器人。

In [9]:
import { ChatAnthropic } from "@langchain/anthropic";
import { SystemMessage, HumanMessage, AIMessage, RemoveMessage } from "@langchain/core/messages";
import { MemorySaver } from "@langchain/langgraph-checkpoint";
import { MessagesAnnotation, StateGraph, START, END, Annotation } from "@langchain/langgraph";
import { v4 as uuidv4 } from "uuid";

const memory = new MemorySaver();

// 我们将添加一个“summary”属性（除了“messages”键之外，
// MessagesAnnotation 已经有）
const GraphAnnotation = Annotation.Root({
  ...MessagesAnnotation.spec,
  summary: Annotation<string>({
    reducer: (_, action) => action,
    default: () => "",
  })
})

// 我们将使用这个模型进行对话和总结
const model = new ChatAnthropic({ model: "claude-3-haiku-20240307" });

// 定义调用模型的逻辑
async function callModel(state: typeof GraphAnnotation.State): Promise<Partial<typeof GraphAnnotation.State>> {
  // 如果存在摘要，我们会将其添加为系统消息
  const { summary } = state;
  let { messages } = state;
  if (summary) {
    const systemMessage = new SystemMessage({
      id: uuidv4(),
      content: `Summary of conversation earlier: ${summary}`
    });
    messages = [systemMessage, ...messages];
  }
  const response = await model.invoke(messages);
  // 我们返回一个对象，因为这将被添加到现有状态中
  return { messages: [response] };
}

// 我们现在定义确定是否结束或总结对话的逻辑
function shouldContinue(state: typeof GraphAnnotation.State): "summarize_conversation" | typeof END {
  const messages = state.messages;
  // 如果消息超过 6 条，那么我们会总结对话内容
  if (messages.length > 6) {
    return "summarize_conversation";
  }
  // 否则我们就可以结束了
  return END;
}

async function summarizeConversation(state: typeof GraphAnnotation.State): Promise<Partial<typeof GraphAnnotation.State>> {
  // 首先我们总结一下谈话内容
  const { summary, messages } = state;
  let summaryMessage: string;
  if (summary) {
    // 如果摘要已存在，我们将使用不同的系统提示符
    // 总结一下，如果没有的话
    summaryMessage = `This is summary of the conversation to date: ${summary}\n\n` +
      "Extend the summary by taking into account the new messages above:";
  } else {
    summaryMessage = "Create a summary of the conversation above:";
  }

  const allMessages = [...messages, new HumanMessage({
    id: uuidv4(),
    content: summaryMessage,
  })];
  const response = await model.invoke(allMessages);
  // 我们现在需要删除不再希望显示的消息
  // 我将删除除最后两条消息之外的所有消息，但您可以更改此设置
  const deleteMessages = messages.slice(0, -2).map((m) => new RemoveMessage({ id: m.id }));
  if (typeof response.content !== "string") {
    throw new Error("Expected a string response from the model");
  }
  return { summary: response.content, messages: deleteMessages };
}

// 定义一个新图
const workflow = new StateGraph(GraphAnnotation)
  // 定义对话节点和摘要节点
  .addNode("conversation", callModel)
  .addNode("summarize_conversation", summarizeConversation)
  // 将入口点设置为对话
  .addEdge(START, "conversation")
  // 我们现在添加一个条件边
  .addConditionalEdges(
    // 首先，我们定义起始节点。我们使用“对话”。
    // 这意味着这些是调用“conversation”节点后获取的边。
    "conversation",
    // 接下来，我们传入将确定接下来调用哪个节点的函数。
    shouldContinue
  )
  // 我们现在添加一条从“summarize_conversation”到 END 的正常边缘。
  // 这意味着在调用“summarize_conversation”之后，我们就结束了。
  .addEdge("summarize_conversation", END);

// 最后，我们编译它！
const app = workflow.compile({ checkpointer: memory });

## 使用图表

In [23]:
const printUpdate = (update: Record<string, any>) => {
  Object.keys(update).forEach((key) => {
    const value = update[key];

    if ("messages" in value && Array.isArray(value.messages)) {
      value.messages.forEach((msg) => {
        console.log(`\n================================ ${msg._getType()} Message =================================`)
        console.log(msg.content);
      })
    }
    if ("summary" in value && value.summary) {
      console.log(value.summary);
    }
  })
}

In [24]:
import { HumanMessage } from "@langchain/core/messages";

const config = { configurable: { thread_id: "4" }, streamMode: "updates" as const }

const inputMessage = new HumanMessage("hi! I'm bob")
console.log(inputMessage.content)
for await (const event of await app.stream({ messages: [inputMessage] }, config)) {
  printUpdate(event)
}

const inputMessage2 = new HumanMessage("What did I sat my name was?")
console.log(inputMessage2.content)
for await (const event of await app.stream({ messages: [inputMessage2] }, config)) {
  printUpdate(event)
}

const inputMessage3 = new HumanMessage("i like the celtics!")
console.log(inputMessage3.content)
for await (const event of await app.stream({ messages: [inputMessage3] }, config)) {
  printUpdate(event)
}

hi! I'm bob

================================ ai Message =================================
Okay, got it. Hello Bob, it's nice to chat with you again. I recognize that you've repeatedly stated your name is Bob throughout our conversation. Please let me know if there is anything I can assist you with.

================================ remove Message =================================


================================ remove Message =================================


================================ remove Message =================================


================================ ai Message =================================
In our conversation, you have stated multiple times that your name is Bob. For example, you said "I'm Bob", "hi! I'm bob", and similar variations where you clearly identified yourself as Bob.
i like the celtics!

================================ ai Message =================================
Ah I see, you mentioned earlier that you like the Boston Celtics basketball t

我们可以看到，到目前为止还没有发生摘要 - 这是因为列表中只有 6 条消息。

In [25]:
const values = (await app.getState(config)).values
console.log(values)

{
  messages: [
    HumanMessage {
      "content": "hi! I'm bob",
      "additional_kwargs": {},
      "response_metadata": {}
    },
    AIMessage {
      "id": "msg_01G6WKqKHK8W371793Hm6eNM",
      "content": "Okay, got it. Hello Bob, it's nice to chat with you again. I recognize that you've repeatedly stated your name is Bob throughout our conversation. Please let me know if there is anything I can assist you with.",
      "additional_kwargs": {
        "id": "msg_01G6WKqKHK8W371793Hm6eNM",
        "type": "message",
        "role": "assistant",
        "model": "claude-3-haiku-20240307",
        "stop_reason": "end_turn",
        "stop_sequence": null,
        "usage": {
          "input_tokens": 579,
          "output_tokens": 50
        }
      },
      "response_metadata": {
        "id": "msg_01G6WKqKHK8W371793Hm6eNM",
        "model": "claude-3-haiku-20240307",
        "stop_reason": "end_turn",
        "stop_sequence": null,
        "usage": {
          "input_tokens": 579,


现在让我们发送另一条消息

In [26]:
const inputMessage4 = new HumanMessage("i like how much they win")
console.log(inputMessage4.content)
for await (const event of await app.stream({ messages: [inputMessage4] }, config)) {
  printUpdate(event)
}

i like how much they win

================================ ai Message =================================
I agree, the Celtics' impressive track record of wins and championships is a big part of what makes them such an iconic and beloved team. Their sustained success over decades is really remarkable. 

Some key reasons why the Celtics have been so dominant:

- Great coaching - They've had legendary coaches like Red Auerbach, Doc Rivers, and Brad Stevens who have led the team to titles.

- Hall of Fame players - Superstars like Bill Russell, Larry Bird, Kevin Garnett, and Paul Pierce have powered the Celtics' championship runs.

- Winning culture - The Celtics have built a winning mentality and tradition of excellence that gets passed down to each new generation of players.

- Loyal fanbase - The passionate Celtics fans pack the stands and provide a strong home court advantage.

The combination of top-tier talent, smart management, and devoted supporters has allowed the Celtics to reign 

如果我们现在检查状态，我们可以看到我们有对话摘要以及最后两条消息

In [27]:
const values2 = (await app.getState(config)).values
console.log(values2)

{
  messages: [
    HumanMessage {
      "content": "i like how much they win",
      "additional_kwargs": {},
      "response_metadata": {}
    },
    AIMessage {
      "id": "msg_01W8C1nXeydqM3E31uCCeJXt",
      "content": "I agree, the Celtics' impressive track record of wins and championships is a big part of what makes them such an iconic and beloved team. Their sustained success over decades is really remarkable. \n\nSome key reasons why the Celtics have been so dominant:\n\n- Great coaching - They've had legendary coaches like Red Auerbach, Doc Rivers, and Brad Stevens who have led the team to titles.\n\n- Hall of Fame players - Superstars like Bill Russell, Larry Bird, Kevin Garnett, and Paul Pierce have powered the Celtics' championship runs.\n\n- Winning culture - The Celtics have built a winning mentality and tradition of excellence that gets passed down to each new generation of players.\n\n- Loyal fanbase - The passionate Celtics fans pack the stands and provide a strong h

我们现在可以继续交谈了！请注意，即使我们只有最后两条消息，我们仍然可以向它询问有关对话中前面提到的事情的问题（因为我们总结了这些）

In [28]:
const inputMessage5 = new HumanMessage("what's my name?");
console.log(inputMessage5.content)
for await (const event of await app.stream({ messages: [inputMessage5] }, config)) {
  printUpdate(event)
}

what's my name?

================================ ai Message =================================
Your name is Bob. You have stated this multiple times throughout our conversation, repeatedly introducing yourself as "Bob" or "I'm Bob".


In [29]:
const inputMessage6 = new HumanMessage("what NFL team do you think I like?");
console.log(inputMessage6.content)
for await (const event of await app.stream({ messages: [inputMessage6] }, config)) {
  printUpdate(event)
}

what NFL team do you think I like?

================================ ai Message =================================
I do not actually have any information about what NFL team you might like. In our conversation so far, you have only expressed that you are a fan of the Boston Celtics basketball team. You have not mentioned any preferences for NFL teams. Without you providing any additional details about your football team allegiances, I do not want to make an assumption about which NFL team you might be a fan of. Could you please let me know if there is an NFL team you particularly enjoy following?


In [30]:
const inputMessage7 = new HumanMessage("i like the patriots!");
console.log(inputMessage7.content)
for await (const event of await app.stream({ messages: [inputMessage7] }, config)) {
  printUpdate(event)
}

i like the patriots!

================================ ai Message =================================
Okay, got it. Based on your latest message, I now understand that in addition to being a fan of the Boston Celtics basketball team, you also like the New England Patriots NFL team.

That makes a lot of sense given that both the Celtics and Patriots are major sports franchises based in the Boston/New England region. It's common for fans to follow multiple professional teams from the same geographic area.

I appreciate you sharing this additional information about your football team preferences. Knowing that you're a Patriots fan provides helpful context about your sports interests and loyalties. It's good for me to have that understanding as we continue our conversation.

Please let me know if there's anything else you'd like to discuss related to the Patriots, the Celtics, or your overall sports fandom. I'm happy to chat more about those topics.

================================ remove M